## Process the titanic_test dataset
In the "M3L2 - Process (Train)" notebook, we did the following adjustments to the titanic_train dataset:
* Dropped the cabin column
* Dropped any paseengers with 'Embarked' data missing
* Replaced index with PassengerId
* Created a Salutation column
* Replaced missing Age data with the median of their Salutation group

We must repeat these steps for the test dataset.  
Note that Kaggle does not provide the Survived column for the test dataset.  This is in order to prevent researchers from tuning their models specifically for the test dataset and thereby artificially inflating their accuracies.  

The code below is an abbridged version of the "M3L2 - Process (Train)" notebook.

In [1]:
# Import libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Workshop Functions
import sys
sys.path.append('..')
from Wksp722_functions import * 

In [2]:
df = pd.read_csv("titanic_test.csv")
df.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [3]:
# reset index to PassengerId
df.set_index('PassengerId', inplace=True)

In [4]:
df.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [5]:
# Replace 1 passenger that had a missing Fare with median

print(df.shape) # size of df before 
df.loc[df.loc[:,'Fare'].isnull(),'Fare'] = df.loc[:,'Fare'].median()
print(df.shape) # size of df after
df.head()

(418, 10)
(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
# Drop Cabin column

df.drop(['Cabin'], axis=1, inplace=True)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


### Create 'Salutation' Column

In [7]:
split_name = df.loc[:,'Name'].str.split(n=3, expand=True)

In [8]:
# let's see the list of salutations
df.loc[:,'Salutation']=split_name[1]

In [9]:
#  Count the number of passengers for each salutation
df.groupby('Salutation').count().loc[:,'Name']

Salutation
Col.         2
Dona.        1
Dr.          1
Master.     21
Miss.       78
Mr.        240
Mrs.        72
Ms.          1
Rev.         2
Name: Name, dtype: int64

It's important to use the median ages calculated during the training set to replace missing ages.  That way, we're using the same substitution values for both sets.  

In [10]:
median_age = pd.read_csv('median_age.csv')
median_age.set_index('Salutation',inplace=True)

In [11]:
# let's remember how may null values there are in the existing Age column
df.loc[:,'Age'].isnull().sum()

86

In [12]:
for ind in df.index:
    if np.isnan(df.loc[ind,'Age']): 
        df.loc[ind,'Age'] = median_age.loc[df.loc[ind,'Salutation'],'Age']

### Save Test dataset

In [13]:
df.to_csv('titanic_test_cleaned.csv')